# Advanced Time Series Econometrics - Assignment

This notebook contains the code for the ATSE assignment. The topics considered are:
- State space switching models
- Markov switching models

Note that the data we are using are annualized and sampled quarterly. Furthermore, the data represents change in the variable in percent points.


In [ ]:
import os
import numpy as np
import pandas as pd
import statsmodels.api as sm
import statsmodels.formula.api as smf
import matplotlib.pyplot as plt

In [ ]:
plt.style.use('ggplot')

## Data

In [ ]:
data = pd.read_csv(filepath_or_buffer='data.csv', sep=';', parse_dates=[0], decimal=',')

In [ ]:
data.info()

In [ ]:
data.head()

In [ ]:
data.set_index('observation_date', drop=True, inplace=True)

In [ ]:
data.info()

In [ ]:
print('PCE: personal consumption')
print('PAYEMS: employment')
print('IPMAN: industrial production')
data.describe()

In [ ]:
plt.plot('PCE', data=data)
plt.plot('PAYEMS', data=data)
plt.plot('IPMAN', data=data)

plt.xlabel('Date')
plt.legend()
plt.tight_layout()

### Split the data

In [ ]:
train = data.copy().iloc[0:189]
test = data.copy().iloc[189:]
print(train.index[0])
print(train.index[-1])
print('-'*100)
print(test.index[0])
print(test.index[-1])
print('-'*100)
print(data.shape)
print(train.shape)
print(test.shape)
print('-'*100)
print(data.index[0])
print(data.index[-1])

## Question 1: Markov Switching Models

### A

In [ ]:
mean_ipman = train['IPMAN'].mean()
sd_ipman = train['IPMAN'].std()

sigma_s1 = 0.5*sd_ipman
sigma_s2 = 1.5*sd_ipman

#### I

In [ ]:
markov_hamilton_1 = sm.tsa.MarkovRegression(endog=train['IPMAN'], k_regimes=2, trend='c', switching_variance=True) # specify the model
print(markov_hamilton_1.param_names)

In [ ]:
markov_hamilton_1.initialize_known([1,0])  # initialize the model in state 1

# fit the model by maximum likelihood using the Hamilton filter using start parameters in the order printed in the output of the cell above
# and don't use EM algorithm to improve starting values by setting em_iter=0 and search_iter=0
markov_hamilton_1_fitted = markov_hamilton_1.fit(start_params=[0.8, 0.2, mean_ipman, mean_ipman, sigma_s1**2, sigma_s2**2], em_iter=0, search_iter=0)

#print summary of fitted model
print(markov_hamilton_1_fitted.summary())

In [ ]:
mh1_params = markov_hamilton_1_fitted.params.copy()
mh1_params

In [ ]:
mh1_initial_probabilities = markov_hamilton_1_fitted.initial_probabilities  # extract the initial probabilities
mh1_initial_probabilities

In [ ]:
mh1_P = markov_hamilton_1_fitted.regime_transition.reshape((2,2))  # extract the left stochastic transition matrix
mh1_P

#### II

In [ ]:
markov_hamilton_2 = sm.tsa.MarkovRegression(endog=train['IPMAN'], k_regimes=2, trend='c', switching_variance=True) # specify the model
print(markov_hamilton_2.param_names)

In [ ]:
markov_hamilton_2.initialize_known([0,1])  # initialize the model in state 2

# fit the model by maximum likelihood using the Hamilton filter using start parameters in the order printed in the output of the cell above
# and don't use EM algorithm to improve starting values by setting em_iter=0 and search_iter=0
markov_hamilton_2_fitted = markov_hamilton_2.fit(start_params=[0.8, 0.2, mean_ipman, mean_ipman, sigma_s1**2, sigma_s2**2], em_iter=0, search_iter=0)

#print summary of fitted model
print(markov_hamilton_2_fitted.summary())

In [ ]:
mh2_params = markov_hamilton_2_fitted.params.copy()
mh2_params

In [ ]:
mh2_params.iloc[-2]**0.5

In [ ]:
mh2_params.iloc[-1]**0.5

In [ ]:
mh2_initial_probabilities = markov_hamilton_2_fitted.initial_probabilities  # extract the initial probabilities
mh2_initial_probabilities

In [ ]:
mh2_P = markov_hamilton_2_fitted.regime_transition.reshape((2,2))  # extract the left stochastic transition matrix
mh2_P

#### III

In [ ]:
markov_hamilton_ss = sm.tsa.MarkovRegression(endog=train['IPMAN'], k_regimes=2, trend='c', switching_variance=True) # specify the model
print(markov_hamilton_ss.param_names)

In [ ]:
markov_hamilton_ss.initialize_steady_state()  # initialize the model in the steady state probabilities

# fit the model by maximum likelihood using the Hamilton filter using start parameters in the order printed in the output of the cell above
# and don't use EM algorithm to improve starting values by setting em_iter=0 and search_iter=0
markov_hamilton_ss_fitted = markov_hamilton_ss.fit(start_params=[0.8, 0.2, mean_ipman, mean_ipman, sigma_s1**2, sigma_s2**2], em_iter=0, search_iter=0)

#print summary of fitted model
print(markov_hamilton_ss_fitted.summary())

In [ ]:
mhss_params = markov_hamilton_ss_fitted.params.copy()
mhss_params

In [ ]:
mhss_initial_probabilities = markov_hamilton_ss_fitted.initial_probabilities  # extract the initial probabilities
mhss_initial_probabilities

In [ ]:
mhss_P = markov_hamilton_ss_fitted.regime_transition.reshape((2,2))  # extract the left stochastic transition matrix
mhss_P

In [ ]:
dir(markov_hamilton_ss_fitted)

In [ ]:
mhss_P

In [ ]:
(1-mhss_P[1,1])/(2-mhss_P[0,0]-mhss_P[1,1])  # steady sate p1

### B

Unfortunately, the *statsmodels* library does not yet support out-of-sample predictions for Markov switching models. Fortunately, this is rather easy to do manually using the formulas from the slides. Note that, while the *statsmodels* library does have a *predict* method for Markov switching models, this method gives an *NotImplementedError* when applied to out-of-sample data.

We are asked to use the model that was initialized in state 1

In [ ]:
markov_hamilton_1_fitted.predicted_marginal_probabilities.head()  # check out the predicted ksi vector

In [ ]:
markov_hamilton_1_fitted.filtered_marginal_probabilities.head()  # check out the updated (filtered) ksi vector

In [ ]:
initial_ksi = markov_hamilton_1_fitted.filtered_marginal_probabilities.iloc[-1]
predicted_ksi = np.array([np.linalg.matrix_power(mh1_P, i) @ initial_ksi for i in range (1,test.shape[0]+1)])
predicted_ksi

In [ ]:
# next we compute the predicted y_t
out_sample_forecasted_y = mh1_params['const[0]'] * predicted_ksi[:,0] + mh1_params['const[1]'] * predicted_ksi[:,1]
out_sample_forecasted_y

In [ ]:
mh1_out_of_sample_forecast_results = pd.DataFrame(test['IPMAN'].copy())
mh1_out_of_sample_forecast_results['predicted_IPMAN'] = out_sample_forecasted_y
mh1_out_of_sample_forecast_results.rename(columns={'IPMAN': 'actual_IPMAN'}, inplace=True)
mh1_out_of_sample_forecast_results['forecast_error'] = mh1_out_of_sample_forecast_results['actual_IPMAN'] - mh1_out_of_sample_forecast_results['predicted_IPMAN']
mh1_out_of_sample_forecast_mse = (mh1_out_of_sample_forecast_results['forecast_error'] ** 2).mean()
print(f'MSFE: {mh1_out_of_sample_forecast_mse}')
mh1_out_of_sample_forecast_results

### C

In [ ]:
markov_hamilton_1_out_sample = sm.tsa.MarkovRegression(endog=data['IPMAN'], k_regimes=2, trend='c', switching_variance=True) # specify the model
markov_hamilton_1_out_sample.initialize_known([1,0])
print(markov_hamilton_1_out_sample.param_names)

In [ ]:
mh1_out_sample_filter_results = markov_hamilton_1_out_sample.filter(params=mh1_params)
mh1_out_sample_predicted_ksi_1_step = mh1_out_sample_filter_results.predicted_marginal_probabilities.tail(14)
mh1_out_sample_predicted_ksi_1_step

In [ ]:
# next we compute the predicted y_t using an expanding sample
out_sample_forecasted_y_1_step = mh1_params['const[0]']*mh1_out_sample_predicted_ksi_1_step.loc[:,0] + mh1_params['const[1]']*mh1_out_sample_predicted_ksi_1_step.loc[:,1]
out_sample_forecasted_y_1_step

In [ ]:
mh1_out_of_sample_forecast_results_1_step = pd.DataFrame(test['IPMAN'].copy())
mh1_out_of_sample_forecast_results_1_step['predicted_IPMAN'] = out_sample_forecasted_y_1_step
mh1_out_of_sample_forecast_results_1_step.rename(columns={'IPMAN': 'actual_IPMAN'}, inplace=True)
mh1_out_of_sample_forecast_results_1_step['forecast_error'] = mh1_out_of_sample_forecast_results_1_step['actual_IPMAN'] - mh1_out_of_sample_forecast_results_1_step['predicted_IPMAN']
mh1_out_of_sample_forecast_mse_1_step = (mh1_out_of_sample_forecast_results_1_step['forecast_error'] ** 2).mean()
print(f'MSFE: {mh1_out_of_sample_forecast_mse_1_step}')
mh1_out_of_sample_forecast_results_1_step

### D

Not implemented here. Look at other notebook

In [ ]:
# TODO: get em estimate of p0 and p1

In [ ]:
# mh_em = sm.tsa.MarkovRegression(endog=train['IPMAN'], k_regimes=2, trend='c', switching_variance=True) # specify the model

In [ ]:
# mh_em.initialize_known([0.5,0.5])  # initialize the model in state 1 and 2 with equal probability
# 
# # fit the model by maximum likelihood using the Hamilton filter using start parameters in the order printed in the output of the cell above
# # NOW we use EM algorithm to improve starting values by setting em_iter=0 and search_iter=0
# mh_em_fitted = mh_em.fit(start_params=[0.8, 0.2, mean_ipman, mean_ipman, sigma_s1**2, sigma_s2**2], em_iter=1000, search_iter=0)
# 
# #print summary of fitted model
# print(mh_em_fitted.summary())

In [ ]:
# mh_em_fitted.predicted_marginal_probabilities

In [ ]:
# mh_em.initial_probabilities(params=mh_em_fitted.params)

In [ ]:
# dir(mh_em_fitted)

In [ ]:
# mh_em_fitted.initial_probabilities

## Question 2: State Space Models

First, we need to demean the data based on the first T=189 observations

In [ ]:
train_demeaned = (train - train.mean()).copy()
train_demeaned.head()

### A

Not implemented here. The code is implemented in the other notebook.

In [ ]:
# ar_1_model = sm.tsa.SARIMAX(train_demeaned['PCE'], order=(1,0,0), trend='n')
# ar_1_model_fitted = ar_1_model.fit()

In [ ]:
# print(ar_1_model_fitted.summary())

In [ ]:
# ar1 =sm.tsa.AutoReg(train_demeaned['PCE'], lags=1, trend='n')
# result = ar1.fit()
# print(result.summary())